In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE185909"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE185909"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE185909.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE185909.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE185909.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Co-expression network analysis of frontal cortex during the progression of Alzheimer’s disease   "
!Series_summary	"Using WGCNA and enrichment analyses to identify pathway level differences between individuals with no cognitive impairment, mild cognitive impairment, and Alzheimer’s disease."
!Series_summary	"Frozen frontal cortex (BA10) tissue from NCI, MCI, and mild/moderate AD cases (n = 12/group) representing both genders was acquired postmortem from participants in the Rush Religious Orders Study, a longitudinal clinical pathologic study of aging and AD in elderly Catholic clergy"
!Series_overall_design	"Nimblegen expression array human frontal cortex - NCI (No Cognative Impairment) vs. MCI (Mild Cognative Impairment) vs. AD (alzheimers disease);  Labeled cDNA was digested and hybridized to NimbleGen 12 x 135K human arrays for 18 hrs at 42°C and analyzed on a GenePix 4200A scanner (Molecular Devices). Probe intensity levels were quantified wit

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, this dataset contains gene expression data from NimbleGen expression array
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Alzheimer's Disease): Row 0 contains diagnosis information
trait_row = 0

# Age: Row 2 contains age information
age_row = 2

# Gender: Row 1 contains gender information
gender_row = 1

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert diagnosis to binary: 1 for AD, 0 for non-AD (NCI, MCI)"""
    if not value or ":" not in value:
        return None
    
    diagnosis = value.split(": ")[1].strip().upper()
    
    if diagnosis == "AD":
        return 1
    elif diagnosis in ["NCI", "MCI"]:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to continuous numeric value"""
    if not value or ":" not in value:
        return None
    
    try:
        age = float(value.split(": ")[1].strip())
        return age
    except:
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for Female, 1 for Male"""
    if not value or ":" not in value:
        return None
    
    gender = value.split(": ")[1].strip().lower()
    
    if gender == "female":
        return 0
    elif gender == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # clinical_data is assumed to be available from a previous step
    clinical_df = geo_select_clinical_features(
        clinical_data,  # Assuming this was created in a previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed data
    print("Preview of clinical data:")
    print(preview_df(clinical_df))
    
    # Save the processed data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM5625602': [1.0, 83.8110882957, 1.0], 'GSM5625603': [1.0, 83.8110882957, 1.0], 'GSM5625604': [0.0, 80.5338809035, 1.0], 'GSM5625605': [0.0, 80.5338809035, 1.0], 'GSM5625606': [0.0, 85.1635865845, 0.0], 'GSM5625607': [0.0, 85.1635865845, 0.0], 'GSM5625608': [1.0, 83.3976728268, 1.0], 'GSM5625609': [1.0, 83.3976728268, 1.0], 'GSM5625610': [0.0, 76.3093771389, 1.0], 'GSM5625611': [1.0, 80.3230663929, 1.0], 'GSM5625612': [1.0, 80.3230663929, 1.0], 'GSM5625613': [1.0, 80.3230663929, 1.0], 'GSM5625614': [1.0, 80.3230663929, 1.0], 'GSM5625615': [0.0, 92.1916495551, 0.0], 'GSM5625616': [0.0, 92.1916495551, 0.0], 'GSM5625617': [0.0, 92.1916495551, 0.0], 'GSM5625618': [1.0, 85.6399726215, 0.0], 'GSM5625619': [1.0, 85.6399726215, 0.0], 'GSM5625620': [1.0, 86.2477754962, 1.0], 'GSM5625621': [1.0, 86.2477754962, 1.0], 'GSM5625622': [1.0, 86.2477754962, 1.0], 'GSM5625623': [1.0, 87.3839835729, 1.0], 'GSM5625624': [1.0, 87.3839835729, 1.0], 'GSM5625625': [0.0, 82.9349760

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['AB000409', 'AB000463', 'AB000781', 'AB001328', 'AB002294', 'AB002308',
       'AB002311', 'AB002313', 'AB002360', 'AB002377', 'AB002381', 'AB002382',
       'AB002384', 'AB003177', 'AB003333', 'AB006589', 'AB006590', 'AB006621',
       'AB006625', 'AB007457'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on my biomedical knowledge, these identifiers appear to be GenBank accession numbers
# (starting with "AB" followed by numbers), not standard human gene symbols.
# Standard human gene symbols would typically be alphabetical like APOE, BRCA1, etc.
# These accession numbers need to be mapped to gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['AB000409', 'AB000463', 'AB000781', 'AB001328', 'AB002294'], 'GB_ACC': ['AB000409', 'AB000463', 'AB000781', 'AB001328', 'AB002294'], 'DESCRIPTION': ['MAP kinase interacting serine/threonine kinase 1', 'SH3-domain binding protein 2', 'kinase non-catalytic C-lobe domain (KIND) containing 1', 'solute carrier family 15 (oligopeptide transporter), member 1', 'zinc finger protein 646']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Re-examine the gene annotation data to understand available fields
print("Gene annotation columns:")
print(gene_annotation.columns.tolist())

# Let's see more rows to better understand the data structure
print("\nMore detailed view of gene annotation:")
print(gene_annotation.head().to_string())

# Based on the DESCRIPTION field containing full gene names, we need to extract gene symbols 
# The 'ID' column contains the identifiers that match with gene expression data

# 2. Create the mapping dataframe using the appropriate columns
id_col = 'ID'
gene_col = 'DESCRIPTION'  # We'll keep this but properly extract symbols from it

# Create the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, id_col, gene_col)

# 3. Apply gene mapping with proper extraction of gene symbols
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols to standard format after mapping
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print the number of genes and preview first genes
print(f"\nNumber of genes after mapping and normalization: {len(gene_data)}")
print("First 10 genes:")
print(gene_data.index[:10])


Gene annotation columns:
['ID', 'GB_ACC', 'DESCRIPTION']

More detailed view of gene annotation:
         ID    GB_ACC                                                    DESCRIPTION
0  AB000409  AB000409               MAP kinase interacting serine/threonine kinase 1
1  AB000463  AB000463                                   SH3-domain binding protein 2
2  AB000781  AB000781         kinase non-catalytic C-lobe domain (KIND) containing 1
3  AB001328  AB001328  solute carrier family 15 (oligopeptide transporter), member 1
4  AB002294  AB002294                                        zinc finger protein 646



Number of genes after mapping and normalization: 1772
First 10 genes:
Index(['A4GALT', 'AAA1', 'AADAC', 'AARD', 'ABCC11', 'ABCC9', 'ABCD1', 'ABCE1',
       'ABI3', 'ABRAXAS2'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (1772, 35)
Normalized gene data saved to ../../output/preprocess/Alzheimers_Disease/gene_data/GSE185909.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM5625602': [1.0, 83.8110882957, 1.0], 'GSM5625603': [1.0, 83.8110882957, 1.0], 'GSM5625604': [0.0, 80.5338809035, 1.0], 'GSM5625605': [0.0, 80.5338809035, 1.0], 'GSM5625606': [0.0, 85.1635865845, 0.0], 'GSM5625607': [0.0, 85.1635865845, 0.0], 'GSM5625608': [1.0, 83.3976728268, 1.0], 'GSM5625609': [1.0, 83.3976728268, 1.0], 'GSM5625610': [0.0, 76.3093771389, 1.0], 'GSM5625611': [1.0, 80.3230663929, 1.0], 'GSM5625612': [1.0, 80.3230663929, 1.0], 'GSM5625613': [1.0, 80.3230663929, 1.0], 'GSM5625614': [1.0, 80.3230663929, 1.0], 'GSM5625615': [0.0, 92.1916495551, 0.0], 'GSM5625616': [0.0, 92.1916495551, 0.0], 'GSM5625617': [0.0, 92.1916495551, 0.0], 'GSM5625618': [1.0, 85.6399726215, 0.0], 'GSM5625619': [1.0, 85.6399726215, 0

Linked data shape after handling missing values: (35, 1775)
Checking for bias in trait distribution...
For the feature 'Alzheimers_Disease', the least common label is '1.0' with 17 occurrences. This represents 48.57% of the dataset.
The distribution of the feature 'Alzheimers_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 81.73442847365001
  50% (Median): 85.6399726215
  75%: 88.63107460645
Min: 72.7063655031
Max: 92.1916495551
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 13 occurrences. This represents 37.14% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset usability: True
Linked data saved to ../../output/preprocess/Alzheimers_Disease/GSE185909.csv
